<a href="https://colab.research.google.com/github/kimjs2513/TensorFlow/blob/master/007_softmax_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

tf.random.set_seed(777)  # for reproducibility

2.3.0


In [7]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1], #one-hot encoding으로 처리 된 것.
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)


# 분류 클래스 개수
nb_classes = 3

W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

print(W,b)


def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b) #tf.nn.softmax -> softmax 구현

print(hypothesis(x_data))

# Softmax onehot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

def cost_fn(X, Y): #cost 함수 구현
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1) #cross entropy cost function 구현. 
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

print(cost_fn(x_data, y_data))

def grad_fn(X, Y): # 경사하강법 구현
    with tf.GradientTape() as tape:
        cost = cost_fn(X, Y)
        grads = tape.gradient(cost, variables) #variable은 W와 b를 의미

        return grads

print(grad_fn(x_data, y_data))

def fit(X, Y, epochs = 2000, verbos = 100):
    optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01)
    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i == 0) or ((i + 1) % verbos) == 0:
            print("Loss at epoch %d : %f" % (i + 1, cost_fn(X, Y).numpy()))
fit(x_data, y_data)  

b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1)) #tf.argmax(2차원배열, 1)을 사용하면 각 행 중에서 가장 큰 수를 찾아낸다. / 1이 아닌 0을 사용하면 각 열에서 가장 큰 수를 찾아냄.
print(tf.argmax(y_data, 1)) # matches with y_data

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-1.2832785 , -0.57372373,  0.8555362 ],
       [ 0.36409333, -1.008571  , -1.1627557 ],
       [ 1.7711345 , -0.5514343 , -0.11697779],
       [ 0.9940506 ,  1.0928452 ,  1.4065602 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([ 0.00882025, -0.35115844,  0.43264598], dtype=float32)>
tf.Tensor(
[[8.6973101e-01 8.5733375e-03 1.2169565e-01]
 [8.4027678e-01 7.0474501e-04 1.5901843e-01]
 [2.1421744e-01 4.4507373e-04 7.8533745e-01]
 [4.8144269e-01 2.1568791e-05 5.1853567e-01]
 [9.9999976e-01 1.4124402e-09 1.8492470e-07]
 [9.9942112e-01 1.4904855e-06 5.7730370e-04]
 [9.9999976e-01 6.0302913e-10 1.9464851e-07]
 [1.0000000e+00 1.6535809e-11 9.6672768e-09]], shape=(8, 3), dtype=float32)
tf.Tensor([[6.5242233e-07 1.2419434e-06 9.9999809e-01]], shape=(1, 3), dtype=float32)
tf.Tensor(6.0906534, shape=(), dtype=float32)
[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.889766  , -0.74857426